<a href="https://colab.research.google.com/github/akash5169/Deep-Learning/blob/main/Audio-Classification/AudioDataPreparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import os
import pandas as pd
from google.colab import drive
import librosa
import math
import json

In [23]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
DATASET_PATH="/content/drive/MyDrive/Data/genres_original"
JSON_PATH="/content/drive/MyDrive/JSON_AUDIO_DATA/data.json"
SAMPLE_RATE=22050
DURATION=30
SAMPLES_PER_TRACK=SAMPLE_RATE* DURATION

In [38]:
def save_mfcc(dataset_path, json_path, n_mfcc=13, n_fft=2048, hop_length=512, num_segments=5):
  #dictionary to store data
  data= {
      "mapping":[],
      "mfcc":[],
      "lables":[]
  }

  num_samples_per_segment= (SAMPLES_PER_TRACK/num_segments)
  expected_num_mfcc_vectors_per_segment = math.ceil(num_samples_per_segment/hop_length)

  #loop through all generes
  for i , (dirpath, dirnames,filenames) in enumerate(os.walk(dataset_path)):
    dirpath_components=dirpath.split("/")
    semantic_label=dirpath_components[-1]
    data["mapping"].append(semantic_label)

    print("\n Processing {}".format(semantic_label))
    print("i=",i-1)
    print(dirnames)
    print(filenames)
    #process files for specific genere
    for f in filenames:
      #load audio file
      file_path=os.path.join(dirpath,f)
      signal,sr=librosa.load(file_path, sr=SAMPLE_RATE)

      #process segments extracting mfcc and storing data
      for s in range(num_segments):
        start_sample= num_samples_per_segment* s
        finish_sample= start_sample+ num_samples_per_segment

        start_sample=int(start_sample)
        finish_sample=int(finish_sample)
        #extract MFCC  
        mfcc=librosa.feature.mfcc(signal[start_sample:finish_sample], sr=sr, n_fft=n_fft, n_mfcc=n_mfcc, hop_length=hop_length)
        mfcc=mfcc.T

        #store mfcc for segment if it has the expected length
        if len(mfcc) == expected_num_mfcc_vectors_per_segment:
          data["mfcc"].append(mfcc.tolist())
          if i<7:
            data["lables"].append(i-1)
          else:
            data["lables"].append(i-2)
          
          print("{}, segment:{}".format(file_path,s+1))
  
  with open(JSON_PATH,"w") as fp:
    json.dump(data,fp,indent=4)          



In [40]:
if __name__ =="__main__":
  save_mfcc(DATASET_PATH, JSON_PATH,num_segments=10)

Streaming output truncated to the last 5000 lines.
/content/drive/MyDrive/Data/genres_original/rock/rock.00007.wav, segment:4
/content/drive/MyDrive/Data/genres_original/rock/rock.00007.wav, segment:5
/content/drive/MyDrive/Data/genres_original/rock/rock.00007.wav, segment:6
/content/drive/MyDrive/Data/genres_original/rock/rock.00007.wav, segment:7
/content/drive/MyDrive/Data/genres_original/rock/rock.00007.wav, segment:8
/content/drive/MyDrive/Data/genres_original/rock/rock.00007.wav, segment:9
/content/drive/MyDrive/Data/genres_original/rock/rock.00007.wav, segment:10
/content/drive/MyDrive/Data/genres_original/rock/rock.00008.wav, segment:1
/content/drive/MyDrive/Data/genres_original/rock/rock.00008.wav, segment:2
/content/drive/MyDrive/Data/genres_original/rock/rock.00008.wav, segment:3
/content/drive/MyDrive/Data/genres_original/rock/rock.00008.wav, segment:4
/content/drive/MyDrive/Data/genres_original/rock/rock.00008.wav, segment:5
/content/drive/MyDrive/Data/genres_original/rock